In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import time
import requests
import time
import sys
from tqdm import tqdm
#Avoid scientific notation
pd.options.display.float_format = '{:.0f}'.format

In [2]:
url1 = f'http://www.omdbapi.com/?apikey=1a06a86a&i=tt0087332'

response = requests.get(url1)
data = response.json()

data

{'Title': 'Ghostbusters',
 'Year': '1984',
 'Rated': 'PG',
 'Released': '08 Jun 1984',
 'Runtime': '105 min',
 'Genre': 'Action, Comedy, Fantasy',
 'Director': 'Ivan Reitman',
 'Writer': 'Dan Aykroyd, Harold Ramis, Rick Moranis',
 'Actors': 'Bill Murray, Dan Aykroyd, Sigourney Weaver',
 'Plot': 'Three parapsychologists forced out of their university funding set up shop as a unique ghost removal service in New York City, attracting frightened yet skeptical customers.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': 'Nominated for 2 Oscars. 9 wins & 9 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMGI0Yjg2ODAtNDYzNi00Njc2LTlkMmMtMmRmYWI5MDE4ZGRkXkEyXkFqcGc@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '95%'},
  {'Source': 'Metacritic', 'Value': '71/100'}],
 'Metascore': '71',
 'imdbRating': '7.8',
 'imdbVotes': '465,733',
 'imdbID': 'tt0087332',
 'Type': 'movie',
 

In [3]:

# https://datasets.imdbws.com/

titles = pd.read_csv('Resources/title.basics.tsv', sep='\t')
ratings = pd.read_csv('Resources/title.ratings.tsv', sep='\t')
# akas = pd.read_csv('Resources/title.akas.tsv', sep='\t')

df = pd.merge(titles, ratings, on="tconst", how="inner")
# main_df = pd.merge(titles,akas, on="tconst", how="inner")
# main_df.head(100)
df.head(10)

/var/folders/sz/6_1pzg752954k81873wl0rkc0000gn/T/ipykernel_71375/1718555477.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('Resources/title.basics.tsv', sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",6,2142
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6,290
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance",6,2179
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5,186
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short,6,2913
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,5,209
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport",5,905
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894,\N,1,"Documentary,Short",5,2288
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5,223
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short",7,7902


In [4]:
#Remove Adult Films
df = df[df['isAdult'] == 0]

#Remove Null values
df = df.loc[(df["startYear"] != '\\N'), :] 
df = df.loc[(df["runtimeMinutes"] != '\\N'), :] #cuts out about 500k
df = df = df.loc[(df["genres"] != '\\N'), :]
df = df.dropna(how="any")

#Change string types to Int where applicable
df['startYear'] = df['startYear'].astype(int)
df['runtimeMinutes'] = df['runtimeMinutes'].astype(int)

df = df[(df['startYear'] >= 2010) & (df['startYear'] <= 2025) ]



# df.loc[(df["startYear"] >= 2010) & (df["startYear"] <= 2025), :]

#Filter to type 'Movie' only. 
df = df[df['titleType'] == 'movie']

# years_runtime_df = years_df.loc[(years_df["Runtime (Min)"] >= 60) & (years_df["Runtime (Min)"] <= 240), :]

testdf = df.copy()

# df.rename(columns={'tconst': 'IMDBid'}, inplace=True)

set_1 = df.copy().reset_index(drop=True)
set_1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,\N,70,Drama,6,229
1,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,7,8268
2,tt0070596,movie,Socialist Realism,El realismo socialista,0,2023,\N,78,"Drama,History",7,137
3,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0,2020,\N,74,"Horror,Music,Thriller",3,1830
4,tt0100275,movie,The Wandering Soap Opera,La Telenovela Errante,0,2017,\N,80,"Comedy,Drama,Fantasy",6,407


In [5]:
#This was used to download the remaining data using the ids of our 129k movies and created the dataframe in the attached file. 

# chunk1 = new_df.loc[0:30000].copy()
# chunk2 = new_df.loc[30001:60000].copy()
# chunk3 = new_df.loc[60001:90000].copy()
# chunk4 = new_df.loc[90001:].copy()


# url= 'http://www.omdbapi.com/?apikey=1a06a86a&i='


# ids = chunk4['IMDBid']
# responses = [] 

# for i in tqdm(ids):
#     try:
#         response = requests.get(url + i)
#         response.raise_for_status()
#         data = response.json()
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching data for {i}: {e}")
#         continue 
#     responses.append(data)
#     time.sleep(0.0005)

# OMDB_DATA = pd.DataFrame(responses)
# OMDB_DATA.to_csv("OMDBDATA4.csv", encoding="utf-8", index=False, header=True)



In [6]:
#Combine outputs into one dataframe and export as CSV. 
combined = pd.concat([pd.read_csv(f'Resources/API_RESPONSE/OMDBDATA{i}.csv') for i in range(1, 5)],axis = 0).reset_index(drop=True)
# combined.to_csv("Resources/set_2.csv", encoding="utf-8", index=False, header=True)




/var/folders/sz/6_1pzg752954k81873wl0rkc0000gn/T/ipykernel_71375/900521904.py:2: DtypeWarning: Columns (1,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  combined = pd.concat([pd.read_csv(f'Resources/API_RESPONSE/OMDBDATA{i}.csv') for i in range(1, 5)],axis = 0).reset_index(drop=True)


In [7]:
combined = combined.drop(['DVD', 'Production','Website','Response','Error','totalSeasons','Season','Episode','seriesID','Poster','Plot'], axis=1)
combined = combined[combined['Country'].str.contains('United States', na=False)]
combined = combined.dropna(subset=['Runtime'])





In [8]:
main = combined.copy().reset_index(drop=True)
main.head()

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,Awards,Ratings,Metascore,imdbRating,imdbVotes,imdbID,Type,BoxOffice
0,The Other Side of the Wind,2018,R,02 Nov 2018,122 min,Drama,Orson Welles,"Orson Welles, Oja Kodar","John Huston, Oja Kodar, Peter Bogdanovich","English, German","France, Iran, United States",9 wins & 8 nominations,"[{'Source': 'Internet Movie Database', 'Value'...",79,7,"8,188",tt0069049,movie,NaN
1,Grizzly II: Revenge,2020,Not Rated,08 Jan 2021,74 min,"Horror, Music, Thriller",André Szöts,"Joan McCall, David Sheldon","George Clooney, Laura Dern, Charlie Sheen",English,United States,NaN,"[{'Source': 'Internet Movie Database', 'Value'...",7,3,"1,801",tt0093119,movie,NaN
2,Mariette in Ecstasy,2019,PG-13,13 Nov 2019,101 min,Drama,John Bailey,Ron Hansen,"Geraldine O'Rawe, Eva Marie Saint, Alex Appel",English,United States,NaN,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,7,72,tt0116991,movie,NaN
3,Deseret,1995,NaN,14 Jun 2014,82 min,Documentary,James Benning,NaN,Fred Gardner,English,United States,NaN,"[{'Source': 'Internet Movie Database', 'Value'...",NaN,6,495,tt0131356,movie,NaN
4,Cooper and Hemingway: The True Gen,2013,Unrated,27 Sep 2013,120 min,Documentary,John Mulholland,John Mulholland,"Sam Waterston, Len Cariou, Nancy Crawford",English,United States,NaN,"[{'Source': 'Internet Movie Database', 'Value'...",54,7,100,tt0159369,movie,NaN


In [9]:
import ast

def parse_ratings(ratings):
    if isinstance(ratings, str):
        try:
            return ast.literal_eval(ratings)
        except Exception as e:
            print("Error parsing ratings:", e)
            return None
    return ratings

main['Ratings'] = main['Ratings'].apply(parse_ratings)

In [10]:
for i in main.index:
   
    for r in main.loc[i,'Ratings']:
        if r['Source'] == 'Rotten Tomatoes':
            main.loc[i,'Rotten_Tomatoes'] =  r['Value'].strip('%')
            break

# df['Released'] = pd.to_datetime(df['Released'], format='%d %b %Y')



In [11]:

main['Released'] = pd.to_datetime(main['Released'], format='%d %b %Y')
main = main.drop(['Type','Year'], axis=1)





# main['BoxOffice'] = main['BoxOffice'].replace('N/A', np.nan)

# main['BoxOffice'] = main['BoxOffice'].str.replace('[\$,]', '', regex=True).astype(int)


In [12]:
main = main.dropna(subset=['Runtime'])
main['Released']

0       2018-11-02
1       2021-01-08
2       2019-11-13
3       2014-06-14
4       2013-09-27
           ...    
39886   2019-03-01
39887   2019-01-30
39888   2022-02-18
39889   2019-04-24
39890   2018-04-22
Name: Released, Length: 39891, dtype: datetime64[ns]

In [13]:
main['R_Year'] = main['Released'].dt.year
main['R_Month'] = main['Released'].dt.month
main['R_Day'] = main['Released'].dt.day



In [14]:
main = main.dropna(subset=['Rated'])

In [15]:
main['Rounded Rating'] = np.round(main['imdbRating'])
# main.groupby('Rounded Rating').count()
main['Rounded Rating'].sum()

106786.0

In [16]:

main['imdbRating'].describe()
#main.groupby('Rounded Rating').count()



count   18354
mean        6
std         1
min         1
25%         5
50%         6
75%         7
max        10
Name: imdbRating, dtype: float64

In [17]:
main[main['imdbRating'] >= 7.4].count() 

Title              2665
Rated              2665
Released           2452
Runtime            2665
Genre              2663
Director           2645
Writer             1908
Actors             2423
Language           2627
Country            2665
Awards             1464
Ratings            2665
Metascore           828
imdbRating         2665
imdbVotes          2665
imdbID             2665
BoxOffice           676
Rotten_Tomatoes     975
R_Year             2452
R_Month            2452
R_Day              2452
Rounded Rating     2665
dtype: int64

In [18]:

main = main[~main['Runtime'].str.contains('S', case=False, na=False)]
main['Runtime'] = main['Runtime'].str.replace(' min', '', regex=False).astype(int)

In [19]:
main = main[main['Runtime'] > 50]
main['Good_Movie'] = main['imdbRating'] > 7.3
main.groupby('Good_Movie').count()

,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,...,Metascore,imdbRating,imdbVotes,imdbID,BoxOffice,Rotten_Tomatoes,R_Year,R_Month,R_Day,Rounded Rating
Good_Movie,,,,,,,,,,,,,,,,,,,,,
False,16357,16357,15575,16357,16349,16281,14955,15999,16105,16357,...,5777,15591,16182,16357,4192,7083,15575,15575,15575,15591
True,2609,2609,2403,2609,2608,2590,1873,2374,2571,2609,...,827,2609,2609,2609,675,974,2403,2403,2403,2609


In [20]:
main

,Title,Rated,Released,Runtime,Genre,Director,Writer,Actors,Language,Country,...,imdbRating,imdbVotes,imdbID,BoxOffice,Rotten_Tomatoes,R_Year,R_Month,R_Day,Rounded Rating,Good_Movie
0,The Other Side of the Wind,R,2018-11-02,122,Drama,Orson Welles,"Orson Welles, Oja Kodar","John Huston, Oja Kodar, Peter Bogdanovich","English, German","France, Iran, United States",...,7,"8,188",tt0069049,NaN,82,2018,11,2,7,False
1,Grizzly II: Revenge,Not Rated,2021-01-08,74,"Horror, Music, Thriller",André Szöts,"Joan McCall, David Sheldon","George Clooney, Laura Dern, Charlie Sheen",English,United States,...,3,"1,801",tt0093119,NaN,9,2021,1,8,3,False
2,Mariette in Ecstasy,PG-13,2019-11-13,101,Drama,John Bailey,Ron Hansen,"Geraldine O'Rawe, Eva Marie Saint, Alex Appel",English,United States,...,7,72,tt0116991,NaN,NaN,2019,11,13,7,True
4,Cooper and Hemingway: The True Gen,Unrated,2013-09-27,120,Documentary,John Mulholland,John Mulholland,"Sam Waterston, Len Cariou, Nancy Crawford",English,United States,...,7,100,tt0159369,NaN,NaN,2013,9,27,7,False
6,Heaven & Hell,TV-MA,2018-11-06,104,Drama,Stuart Paul,Stuart Paul,"Cheryl M. Lynn, Chase Nash, Burt Ward",English,United States,...,3,151,tt0192528,NaN,NaN,2018,11,6,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39875,VFW,Not Rated,2020-02-14,92,"Action, Crime, Horror",Joe Begos,"Max Brallier, Matthew McArdle","Stephen Lang, William Sadler, Fred Williamson",English,United States,...,6,"9,831",tt9894470,NaN,85,2020,2,14,6,False
39876,India Sweets and Spices,PG-13,2022-02-03,101,Comedy,Geeta Malik,Geeta Malik,"Sophia Ali, Deepti Gupta, Rish Shah",English,United States,...,6,893,tt9896876,"$288,714",83,2022,2,3,6,False
39877,Pilgrim's Progress,PG,2019-04-18,108,"Animation, Adventure, Family",Robert Fernandez,"John Bunyan, Robert Fernandez","David Thorpe, John Rhys-Davies, Kristyn Getty",English,United States,...,6,921,tt9896916,"$1,294,596",60,2019,4,18,6,False
39880,Coffee & Kareem,TV-MA,2020-04-03,88,"Action, Comedy, Crime",Michael Dowse,Shane Mack,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",English,United States,...,5,"14,780",tt9898858,NaN,21,2020,4,3,5,False


In [21]:
main['Rotten_Tomatoes'] = main['Rotten_Tomatoes'].astype(float)
main = main.drop(['Ratings','Writer'], axis=1)

In [22]:
final = main.copy().reset_index(drop=True)
final['Genre'] = final['Genre'].fillna('')
for row in tqdm(final.index):
    genres = final.loc[row,'Genre'].split(",")
    for g in genres:
        final.loc[row, g.strip().upper()] = 1
    time.sleep(.001)


100%|████████████████████████████████████| 18966/18966 [00:42<00:00, 442.10it/s]


In [23]:
final.groupby('HISTORY').count()

,Title,Rated,Released,Runtime,Genre,Director,Actors,Language,Country,Awards,...,FAMILY,SPORT,WAR,MUSICAL,WESTERN,NEWS,REALITY-TV,,GAME-SHOW,TALK-SHOW
HISTORY,,,,,,,,,,,,,,,,,,,,,
1,787,787,748,787,787,783,719,779,787,499,...,18,18,41,2,7,50,0,0,0,0


In [24]:
final

,Title,Rated,Released,Runtime,Genre,Director,Actors,Language,Country,Awards,...,HISTORY,SPORT,WAR,MUSICAL,WESTERN,NEWS,REALITY-TV,,GAME-SHOW,TALK-SHOW
0,The Other Side of the Wind,R,2018-11-02,122,Drama,Orson Welles,"John Huston, Oja Kodar, Peter Bogdanovich","English, German","France, Iran, United States",9 wins & 8 nominations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grizzly II: Revenge,Not Rated,2021-01-08,74,"Horror, Music, Thriller",André Szöts,"George Clooney, Laura Dern, Charlie Sheen",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mariette in Ecstasy,PG-13,2019-11-13,101,Drama,John Bailey,"Geraldine O'Rawe, Eva Marie Saint, Alex Appel",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cooper and Hemingway: The True Gen,Unrated,2013-09-27,120,Documentary,John Mulholland,"Sam Waterston, Len Cariou, Nancy Crawford",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Heaven & Hell,TV-MA,2018-11-06,104,Drama,Stuart Paul,"Cheryl M. Lynn, Chase Nash, Burt Ward",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18961,VFW,Not Rated,2020-02-14,92,"Action, Crime, Horror",Joe Begos,"Stephen Lang, William Sadler, Fred Williamson",English,United States,2 wins & 1 nomination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18962,India Sweets and Spices,PG-13,2022-02-03,101,Comedy,Geeta Malik,"Sophia Ali, Deepti Gupta, Rish Shah",English,United States,1 win total,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18963,Pilgrim's Progress,PG,2019-04-18,108,"Animation, Adventure, Family",Robert Fernandez,"David Thorpe, John Rhys-Davies, Kristyn Getty",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18964,Coffee & Kareem,TV-MA,2020-04-03,88,"Action, Comedy, Crime",Michael Dowse,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",English,United States,1 win & 1 nomination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final['BoxOffice'] = final['BoxOffice'].str.replace('$', '', regex=False)
final['BoxOffice'] = final['BoxOffice'].str.replace(',', '', regex=False).astype(float)

In [26]:
final

,Title,Rated,Released,Runtime,Genre,Director,Actors,Language,Country,Awards,...,HISTORY,SPORT,WAR,MUSICAL,WESTERN,NEWS,REALITY-TV,,GAME-SHOW,TALK-SHOW
0,The Other Side of the Wind,R,2018-11-02,122,Drama,Orson Welles,"John Huston, Oja Kodar, Peter Bogdanovich","English, German","France, Iran, United States",9 wins & 8 nominations,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grizzly II: Revenge,Not Rated,2021-01-08,74,"Horror, Music, Thriller",André Szöts,"George Clooney, Laura Dern, Charlie Sheen",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mariette in Ecstasy,PG-13,2019-11-13,101,Drama,John Bailey,"Geraldine O'Rawe, Eva Marie Saint, Alex Appel",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cooper and Hemingway: The True Gen,Unrated,2013-09-27,120,Documentary,John Mulholland,"Sam Waterston, Len Cariou, Nancy Crawford",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Heaven & Hell,TV-MA,2018-11-06,104,Drama,Stuart Paul,"Cheryl M. Lynn, Chase Nash, Burt Ward",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18961,VFW,Not Rated,2020-02-14,92,"Action, Crime, Horror",Joe Begos,"Stephen Lang, William Sadler, Fred Williamson",English,United States,2 wins & 1 nomination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18962,India Sweets and Spices,PG-13,2022-02-03,101,Comedy,Geeta Malik,"Sophia Ali, Deepti Gupta, Rish Shah",English,United States,1 win total,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18963,Pilgrim's Progress,PG,2019-04-18,108,"Animation, Adventure, Family",Robert Fernandez,"David Thorpe, John Rhys-Davies, Kristyn Getty",English,United States,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18964,Coffee & Kareem,TV-MA,2020-04-03,88,"Action, Comedy, Crime",Michael Dowse,"Ed Helms, Taraji P. Henson, Terrence Little Ga...",English,United States,1 win & 1 nomination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
final['BoxOffice']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
          ...  
18961       NaN
18962    288714
18963   1294596
18964       NaN
18965   4588389
Name: BoxOffice, Length: 18966, dtype: float64

In [28]:
import cpi

# Update CPI data from the Bureau of Labor Statistics:
cpi.update()

# Inflate $100 from the year 2000 to its 2020 equivalent:
adjusted_value = cpi.inflate(100, 2000, to=2020)
print(adjusted_value)


150.2967479674797


In [29]:
adjusted_value = cpi.inflate(100, 2001, to=2020)
print(adjusted_value)

146.13833992094862


In [30]:
final = final.dropna(subset=['R_Year'])

In [31]:


for row in tqdm(final.index):
    boxoffice = final.loc[row,'BoxOffice']
    year = final.loc[row,'R_Year']
    if pd.isna(year) or pd.isna(boxoffice):
        continue
    if year < 2024:
        final.loc[row,'BoxOffice_Inf_Adj'] = cpi.inflate(boxoffice, int(year), to=2024)
        time.sleep(.001)
    else:
        continue

  0%|                                                 | 0/17978 [00:00<?, ?it/s]/var/folders/sz/6_1pzg752954k81873wl0rkc0000gn/T/ipykernel_71375/2371488882.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final.loc[row,'BoxOffice_Inf_Adj'] = cpi.inflate(boxoffice, int(year), to=2024)
 15%|█████▊                                | 2773/17978 [02:14<12:15, 20.68it/s]


KeyboardInterrupt: 

In [ ]:
final[['BoxOffice','BoxOffice_Inf_Adj']]


In [ ]:
director = final[['Director','Title']].groupby('Director').count().sort_values(by = 'Title',ascending = False)
comma_check = final[final['Director'].str.contains(',', na=False)]
comma_check

In [ ]:
# final.to_csv("Cleaned.csv", encoding="utf-8", index=False, header=True)
             

In [ ]:
model_data = final.copy()

In [ ]:
model_data.columns


In [ ]:
model_data = model_data[['imdbID','Title','Director','Rated','R_Year','R_Month','R_Day','Runtime','Good_Movie',
       'DRAMA', 'HORROR', 'MUSIC', 'THRILLER',
       'DOCUMENTARY', 'ANIMATION', 'ACTION', 'ADVENTURE', 'BIOGRAPHY',
       'COMEDY', 'FANTASY', 'MYSTERY', 'ROMANCE', 'SCI-FI', 'CRIME', 'FAMILY',
       'HISTORY', 'SPORT', 'WAR', 'MUSICAL', 'WESTERN', 'NEWS']]


In [ ]:
dummies = pd.get_dummies(model_data, columns=['Rated','R_Year','R_Month','R_Day','Director'])
dummies